# Outline:


*   **Notebook 1**:

  *   An overview of Federated Learning (FL)
  *   An overview of Whole Slide Image (WSI) data
  *   Kidney WSI data inspection and creation for HFL task
  *   Baseline experiment on kidney patch classification
  *   An experiment setup for comparing HFL against baseline


*   **Notebook 2**:

  *   A working example of Horizontal Federated Learning (HFL) using TensorFlow Federated on histopathology Kidney images
      *   Tensorflow Federated dataset
      *   Tensorflow Federated model
      *   Tensorflow Federated computations for initialization train and validation
  *   Comparison of FederatedAveraging against the baseline scenario

# Notebook 2:
## A working example of Horizontal Federated Learning (HFL) using TensorFlow Federated on histopathology Kidney images


### Installation of packages

In [24]:
# !pip install tensorflow-federated==0.18
# !pip install nest_asyncio

### Importing Required Packages:

In [3]:
import nest_asyncio

nest_asyncio.apply()

import collections
import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD

2021-11-01 08:52:27.903624: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-01 08:52:27.903655: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Data Loading:

To load the necessary files, specify the "path" variable to be the path to the folder that contains the following files:  
* HFL_kidney.csv
* HFL_dict_data.npy 
* tissue_source_site_data.xlsx
* history_fed_train_id_0_2_3_test_id_1.npy
* my_history_train_id_0_test_id_1.npy
* my_history_train_id_2_test_id_1.npy
* my_history_train_id_3_test_id_1.npy

We already created a dictionary with for key and value pairs for kideny images from the four mentioned hospitals. Each value in dictionary is a tuple of normalized images (normalized to [0-1]  range) and the associated labels for possible three disease types.

In [4]:
# Base path to the folder that contains the following files:  HFL_kidney.csv, HFL_dict_data.npy,  tissue_source_site_data.xlsx, history_fed_train_id_0_2_3_test_id_1.npy,  my_history_train_id_0_test_id_1.npy, my_history_train_id_2_test_id_1.npy, and my_history_train_id_3_test_id_1.npy.
path = "/ssd003/projects/pets/datasets/kidney_histopathology"
# Load the meta-data of Kidney dataset
df = pd.read_csv(path + "/HFL_kidney.csv")
data = np.load(
    path + "/HFL_dict_data.npy", allow_pickle="TRUE"
).item()  # data[0] return the image-lable tuple for hospital number 0

## Creating Federated Data:
The function `tff.simulation.ClientData.from_clients_and_fn`, requires that we write a function that accepts a `client_id` as input and returns a `tf.data.Dataset`. Let's do that in the helper function below: 

In [5]:
batch_size = 64
SHUFFLE_BUFFER = 128


def create_tf_dataset_for_client_fn(client_id):
    client_data = data[client_id]
    dataset = tf.data.Dataset.from_tensor_slices(
        (client_data[0], client_data[1])
    ).prefetch(
        buffer_size=128
    )  # client_data[0] is images,   client_data[1] is labels
    dataset = dataset.shuffle(2000, reshuffle_each_iteration=True).batch(batch_size)

    return dataset

In [6]:
print(tff.__version__)

0.19.0


Now, let's create the training and testing federated data. There are 4 hospitals in our data. We use data of 3 hospitals for training and 1 hospital for testing.

In [7]:
train_client_ids = [0, 2, 3]
test_client_ids = [1]

if tff.__version__ < "0.19.0":

    train_data = tff.simulation.ClientData.from_clients_and_fn(
        client_ids=train_client_ids,
        create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn,
    )

    test_data = tff.simulation.ClientData.from_clients_and_fn(
        client_ids=test_client_ids,
        create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn,
    )
else:
    train_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=train_client_ids,
        serializable_dataset_fn=create_tf_dataset_for_client_fn,
    )

    test_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=test_client_ids,
        serializable_dataset_fn=create_tf_dataset_for_client_fn,
    )

2021-11-01 08:52:55.147628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-01 08:52:55.147741: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-11-01 08:52:55.147814: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-11-01 08:52:55.147891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2021-11-01 08:52:55.147968: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Let's see number of the clients for training federated data and also the sructure of data:

In [8]:
len(train_data.client_ids)

3

In [9]:
train_data.element_type_structure

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float16, name=None),
 TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))

To see exactly how one batch od data look like we create the following example dataset from one the client ids of train federated data:

In [10]:
example_dataset = train_data.create_tf_dataset_for_client(train_data.client_ids[0])
# print(example_dataset)
example_element = iter(example_dataset).next()
print(example_element)

2021-11-01 08:53:02.014661: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1414361088 exceeds 10% of free system memory.
2021-11-01 08:53:03.630183: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1414361088 exceeds 10% of free system memory.


(<tf.Tensor: shape=(64, 224, 224, 3), dtype=float16, numpy=
array([[[[0.651  , 0.596  , 0.7295 ],
         [0.745  , 0.682  , 0.816  ],
         [0.745  , 0.655  , 0.8037 ],
         ...,
         [0.792  , 0.686  , 0.8394 ],
         [0.859  , 0.757  , 0.9136 ],
         [0.863  , 0.7686 , 0.9253 ]],

        [[0.7295 , 0.6665 , 0.8    ],
         [0.82   , 0.753  , 0.886  ],
         [0.7725 , 0.678  , 0.816  ],
         ...,
         [0.741  , 0.6353 , 0.788  ],
         [0.816  , 0.714  , 0.8706 ],
         [0.8354 , 0.741  , 0.898  ]],

        [[0.8037 , 0.7295 , 0.859  ],
         [0.863  , 0.7803 , 0.9097 ],
         [0.8237 , 0.718  , 0.851  ],
         ...,
         [0.741  , 0.643  , 0.792  ],
         [0.8037 , 0.714  , 0.863  ],
         [0.792  , 0.71   , 0.859  ]],

        ...,

        [[0.4236 , 0.2627 , 0.3687 ],
         [0.3843 , 0.2079 , 0.3098 ],
         [0.4236 , 0.2118 , 0.302  ],
         ...,
         [0.6626 , 0.5176 , 0.8076 ],
         [0.718  , 0.5767 , 

We now have almost all the building blocks in place to construct federated datasets.

One of the ways to feed federated data to TFF in a simulation is simply as a Python list, with each element of the list holding the data of an individual user, as a `tf.data.Dataset`. Since we already have an interface for that, let's use it.

The helper function `make_federated_data` below will construct a list of datasets from the
given set of users as an input to a round of training or evaluation.

In [11]:
def make_federated_data(client_data, client_ids):
    return [client_data.create_tf_dataset_for_client(x) for x in client_ids]

In [12]:
federated_train_data = make_federated_data(train_data, train_client_ids)
print("Number of client datasets: {l}".format(l=len(federated_train_data)))
print("First dataset: {d}".format(d=federated_train_data[0]))

2021-11-01 08:53:04.873689: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1414361088 exceeds 10% of free system memory.


Number of client datasets: 3
First dataset: <BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float16, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>


## Tensorflow Federated model
Firat we create a model with Keras (which is the same CNN model that we created in notebook 1):


In [13]:
num_classes = 3
img_height = 224
img_width = 224


def create_keras_model():
    model = Sequential(
        [
            layers.Conv2D(
                16,
                3,
                padding="same",
                activation="relu",
                input_shape=(img_height, img_width, 3),
            ),
            layers.MaxPooling2D(),
            layers.Conv2D(32, 3, padding="same", activation="relu"),
            layers.MaxPooling2D(),
            layers.Conv2D(64, 3, padding="same", activation="relu"),
            layers.MaxPooling2D(),
            layers.Flatten(),
            layers.Dense(128, activation="relu"),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )

    return model

Note that we do not compile the model yet. The loss, metrics, and optimizers are introduced later.

If you have a Keras model like the one we've just defined above, you can have TFF wrap it for you by invoking
`tff.learning.from_keras_model`, passing the model and a sample data batch as
arguments (`input_spec=example_dataset.element_spec`), as shown below.


In [14]:
# We _must_ create a new model here, and _not_ capture it from an external
# scope. TFF will call this within different graph contexts.
def model_fn():
    keras_model = create_keras_model()

    return tff.learning.from_keras_model(
        keras_model,
        input_spec=example_dataset.element_spec,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[
            tf.keras.metrics.CategoricalAccuracy(),
            tf.keras.metrics.AUC(name="AUC"),
        ],
    )

The model_fn is a no-arg function that returns a `tff.learning.Model`. 

### Training the model on federated data 

Now that we have a model wrapped as `tff.learning.Model` for use with TFF, we
can let TFF construct a **Federated Averaging** algorithm by invoking the helper
function `tff.learning.build_federated_averaging_process`, as follows.

One critical note on the Federated Averaging algorithm below, there are **2**
optimizers: a _client_optimizer_ and a _server_optimizer_. The
_client_optimizer_ is only used to compute local model updates on each client.
The _server_optimizer_ applies the averaged update to the global model at the
server. In particular, this means that the choice of optimizer and learning rate
used may need to be different than the ones you have used to train the model on
a standard i.i.d. dataset.



In [15]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.05),
)

In this case, the two computations generated and packed into iterative_process implement Federated Averaging.

Let's start with the `initialize` computation. As is the case for all federated
computations, you can think of it as a function. The computation takes no
arguments, and returns one result - the representation of the state of the
Federated Averaging process on the server. While we don't want to dive into the
details of TFF, it may be instructive to see what this state looks like. You can
visualize it as follows.

In [16]:
str(iterative_process.initialize.type_signature)

'( -> <model=<trainable=<float32[3,3,3,16],float32[16],float32[3,3,16,32],float32[32],float32[3,3,32,64],float32[64],float32[50176,128],float32[128],float32[128,3],float32[3]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

While the above type signature may at first seem a bit cryptic, you can recognize that the server state consists of a model (the initial model parameters that will be distributed to all devices), and optimizer_state (additional information maintained by the server, such as the number of rounds to use for hyperparameter schedules, etc.).

Let's invoke the initialize computation to construct the server state.

In [17]:
state = iterative_process.initialize()

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [18]:
state

ServerState(model=ModelWeights(trainable=[array([[[[-0.0737638 , -0.13577285, -0.08404987,  0.00889607,
          -0.11784039,  0.03777151, -0.14651245,  0.08769804,
          -0.09507216, -0.11193122, -0.05520914, -0.04532318,
          -0.04571591, -0.02778354, -0.15514252,  0.05023652],
         [-0.10695267,  0.07862717, -0.05696589,  0.08383757,
          -0.11863481,  0.0966939 , -0.00148244,  0.01095168,
           0.01516931, -0.18076189, -0.12894003, -0.17812093,
           0.12483078,  0.16109458,  0.10910019,  0.08687136],
         [-0.10040285,  0.18179455,  0.12218845,  0.04833634,
          -0.17279363,  0.02375178, -0.13042332, -0.08673581,
          -0.11684738, -0.02370672,  0.1334678 ,  0.04146707,
          -0.00792222, -0.1585161 , -0.03454764,  0.04671326]],

        [[ 0.07408416, -0.11776005, -0.09453119, -0.00497833,
          -0.06414607, -0.05376828, -0.05157119, -0.02362178,
           0.05239297, -0.05044706, -0.08821932,  0.08170125,
           0.00594722, 

The second of the pair of federated computations, `next`, represents a single
round of Federated Averaging, which consists of pushing the server state
(including the model parameters) to the clients, on-device training on their
local data, collecting and averaging model updates, and producing a new updated
model at the server.

Conceptually, you can think of `next` as having a functional type signature that
looks as follows.

```
SERVER_STATE, FEDERATED_DATA -> SERVER_STATE, TRAINING_METRICS
```

In particular, one should think about `next()` not as being a function that runs on a server, but rather being a declarative functional representation of the entire decentralized computation - some of the inputs are provided by the server (`SERVER_STATE`), but each participating device contributes its own local dataset.

To run a single round of training and visualizing the results We can use the following line of code and federated data we've already generated above:






In [19]:
state, metrics = iterative_process.next(state, federated_train_data)
print("round  1, metrics={}".format(metrics))

round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('categorical_accuracy', 0.6694665), ('AUC', 0.82650024), ('loss', 0.7931211)])), ('stat', OrderedDict([('num_examples', 6429)]))])


## Evaluation
To perform evaluation on federated data, you can construct another *federated
computation* designed for just this purpose, using the
`tff.learning.build_federated_evaluation` function, and passing in your model
constructor as an argument. Note that as the evaluation doesn't perform gradient descent, and there's no need to construct
optimizers.

For experimentation and research, when a centralized test dataset is available,
[Federated Learning for Text Generation](federated_learning_for_text_generation.ipynb)
demonstrates another evaluation option: taking the trained weights from
federated learning, applying them to a standard Keras model, and then simply
calling `tf.keras.models.Model.evaluate()` on a centralized dataset.

In [20]:
evaluation = tff.learning.build_federated_evaluation(
    model_fn
)  # iterative_process = tff.learning.build_federated_averaging_process(model_fn,client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01), server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.05))


federated_test_data = make_federated_data(test_data, test_client_ids)

val_metrics = evaluation(state.model, federated_test_data)

val_metrics

OrderedDict([('eval',
              OrderedDict([('categorical_accuracy', 0.15973742),
                           ('AUC', 0.22238787),
                           ('loss', 1.1567367)])),
             ('stat', OrderedDict([('num_examples', 1828)]))])

In [21]:
a = iter(federated_test_data[0]).next()
print(a[0].shape)

(64, 224, 224, 3)


## Training and evaluation multiple rounds:

Here we run the federated learning algorithm multiple rounds on training clients and evaluate the performance on test client.

First we initialize the model:

In [22]:
state = iterative_process.initialize()

Now let's train and evaluate multiple times

In [23]:
NUM_ROUNDS = 50

loss = list()
accuracy = list()
AUC = list()


val_loss = list()
val_accuracy = list()
val_AUC = list()


evaluation = tff.learning.build_federated_evaluation(model_fn)
federated_test_data = make_federated_data(test_data, test_client_ids)

for round_num in range(1, NUM_ROUNDS + 1):
    state, metrics = iterative_process.next(state, federated_train_data)
    val_metrics = evaluation(state.model, federated_test_data)

    my_loss = metrics["train"]["loss"]
    loss.append(metrics["train"]["loss"])

    my_acc = metrics["train"]["categorical_accuracy"]
    accuracy.append(metrics["train"]["categorical_accuracy"])

    my_AUC = metrics["train"]["AUC"]
    AUC.append(my_AUC)

    my_val_loss = val_metrics["loss"]
    val_loss.append(val_metrics["loss"])

    my_val_accuracy = val_metrics["categorical_accuracy"]
    val_accuracy.append(val_metrics["categorical_accuracy"])

    my_val_AUC = val_metrics["AUC"]
    val_AUC.append(my_val_AUC)

    print(
        f"round: {round_num:2d}, training_loss: {my_loss}, training_accuracy: {my_acc}, train_auc: {my_AUC}, test_loss: {my_val_loss}, test_accuracy: {my_val_accuracy},  test_auc: {my_val_AUC}"
    )


history_fed = {
    "loss": loss,
    "categorical_accuracy": accuracy,
    "AUC": AUC,
    "val_loss": val_loss,
    "val_categorical_accuracy": val_accuracy,
    "val_AUC": val_AUC,
}


# np.save('/content/gdrive/Shareddrives/AI Engineering/PETs/Privacy-Sobhan Hemati/Kidney/history_fed_train_id_0_2_3_test_id_1.npy',history_fed)

KeyboardInterrupt: 

## An experiment setup for comparing HFL against baseline

### Train on one hospital and test on another one.
*   We choose hospitals 0,2,3 as training hospitals and hospital 1 as the test hospital
*   For the baseline scenario, we separately train 3 models on each one of 0, 2, and 3 hospitals and test the three models on data of hospital 1
*   For the FL scenario, we train a shared global model on hospitals 0,2, and 3 while clients keep their training data private
*   Then, we compare the performance of the FL model against the baseline

## Comparison of FederatedAveraging against the baseline scenario

In [ ]:
history_fed = np.load(
    path + "/history_fed_train_id_0_2_3_test_id_1.npy", allow_pickle=True
).item()

In [ ]:
train_id = 3
test_id = 1
history = {}
history["{},{}".format(train_id, test_id)] = np.load(
    path + "/my_history_train_id_{}_test_id_{}.npy".format(train_id, test_id),
    allow_pickle="TRUE",
).item()

In [ ]:
train_id = 2
test_id = 1
history["{},{}".format(train_id, test_id)] = np.load(
    path + "/my_history_train_id_{}_test_id_{}.npy".format(train_id, test_id),
    allow_pickle="TRUE",
).item()

In [ ]:
train_id = 0
test_id = 1
history["{},{}".format(train_id, test_id)] = np.load(
    path + "/my_history_train_id_{}_test_id_{}.npy".format(train_id, test_id),
    allow_pickle="TRUE",
).item()

In [ ]:
fig1 = plt.figure(figsize=(30, 8))
fig1.subplots_adjust(top=0.99, bottom=0.01, hspace=0.3, wspace=0.3)
ax1 = fig1.add_subplot(2, 3, 1)
ax1.plot(
    history_fed["AUC"],
    color="black",
    label="Train AUC on Hispital 1 with Federated Model trained on Hospitals 0-2-3",
)
ax1.plot(
    history["3,1"]["AUC"],
    "*-",
    label="Train AUC on Hispital 1 with Model trained on Hospital 3",
)
ax1.plot(
    history["2,1"]["AUC"],
    "--",
    label="Train AUC on Hispital 1 with Model trained on Hospital 2",
)
ax1.plot(
    history["0,1"]["AUC"],
    "-.",
    label="Train AUC on Hispital 1 with Model trained on Hospital 0",
)
ax1.legend(loc="lower right")
ax1.set_ylabel("Auc")
ax1.set_ylim([0, 1])
ax1.set_title("Train Auc")
ax1.set_xlabel("Epoch/Round")


ax2 = fig1.add_subplot(2, 3, 2)
ax2.plot(
    history_fed["categorical_accuracy"],
    color="black",
    label="Train Accuracy on Hispital 1 with Federated Model trained on Hospitals 0-2-3",
)
ax2.plot(
    history["3,1"]["categorical_accuracy"],
    "*-",
    label="Train Accuracy on Hispital 1 with Model trained on Hospital 3",
)
ax2.plot(
    history["2,1"]["categorical_accuracy"],
    "--",
    label="Train Accuracy on Hispital 1 with Model trained on Hospital 2",
)
ax2.plot(
    history["0,1"]["categorical_accuracy"],
    "-.",
    label="Train Accuracy on Hispital 1 with Model trained on Hospital 0",
)
ax2.legend(loc="lower right")
ax2.set_ylabel("Accuracy")
ax2.set_ylim([0, 1])
ax2.set_title("Train Accuracy")
ax2.set_xlabel("Epoch/Round")

ax3 = fig1.add_subplot(2, 3, 3)
ax3.plot(
    history_fed["loss"],
    color="black",
    label="Train Loss on Hispital 1 with Federated Model trained on Hospitals 0-2-3",
)
ax3.plot(
    history["3,1"]["loss"],
    "*-",
    label="Train Loss of Hispital 1 with Model trained on Hospital 3",
)
ax3.plot(
    history["2,1"]["loss"],
    "--",
    label="Train Loss of Hispital 1 with Model trained on Hospital 2",
)
ax3.plot(
    history["0,1"]["loss"],
    "-.",
    label="Train Loss of Hispital 1 with Model trained on Hospital 0",
)

ax3.legend(loc="lower right")
ax3.set_ylabel("Cross Entropy")
ax3.set_ylim([0, max(ax2.get_ylim()) + 0.5])
ax3.set_title("Train Loss")
ax3.set_xlabel("Epoch/Round")


ax4 = fig1.add_subplot(2, 3, 4)
ax4.plot(
    history_fed["val_AUC"],
    color="black",
    label="Test AUC on Hispital 1 with Federated Model trained on Hospitals 0-2-3",
)
ax4.plot(
    history["3,1"]["val_AUC"],
    "*-",
    label="Test AUC on Hispital 1 with Model trained on Hospital 3",
)
ax4.plot(
    history["2,1"]["val_AUC"],
    "--",
    label="Test AUC on Hispital 1 with Model trained on Hospital 2",
)
ax4.plot(
    history["0,1"]["val_AUC"],
    "-.",
    label="Test AUC on Hispital 1 with Model trained on Hospital 0",
)
ax4.legend(loc="lower right")
ax4.set_ylabel("Auc")
ax4.set_ylim([0, 1])
ax4.set_title("Test Auc")
ax4.set_xlabel("Epoch/Round")


ax5 = fig1.add_subplot(2, 3, 5)
ax5.plot(
    history_fed["val_categorical_accuracy"],
    color="black",
    linewidth=3,
    label="Test Accuracy on Hispital 1 with Federated Model trained on Hospitals 0-2-3",
)
ax5.plot(
    history["3,1"]["val_categorical_accuracy"],
    "*-",
    label="Test Accuracy on Hispital 1 with Model trained on Hospital 3",
)
ax5.plot(
    history["2,1"]["val_categorical_accuracy"],
    "--",
    label="Test Accuracy on Hispital 1 with Model trained on Hospital 2",
)
ax5.plot(
    history["0,1"]["val_categorical_accuracy"],
    "-.",
    label="Test Accuracy on Hispital 1 with Model trained on Hospital 0",
)
ax5.legend(loc="lower right")
ax5.set_ylabel("Accuracy")
ax5.set_ylim([0, 1])
ax5.set_title("Test Accuracy")
ax5.set_xlabel("Epoch/Round")

ax6 = fig1.add_subplot(2, 3, 6)

ax6.plot(
    history_fed["val_loss"],
    color="black",
    label="Test Loss on Hispital 1 with Federated Model trained on Hospitals 0-2-3",
)
ax6.plot(
    history["3,1"]["val_loss"],
    "*-",
    label="Test Loss of Hispital 1 with Model trained on Hospital 3",
)
ax6.plot(
    history["2,1"]["val_loss"],
    "--",
    label="Test Loss of Hispital 1 with Model trained on Hospital 2",
)
ax6.plot(
    history["0,1"]["val_loss"],
    "-.",
    label="Test Loss of Hispital 1 with Model trained on Hospital 0",
)

ax6.legend(loc="lower right")
ax6.set_ylabel("Cross Entropy")
ax6.set_ylim([0, max(ax2.get_ylim()) + 0.5])
ax6.set_title("Test Loss")
ax6.set_xlabel("Epoch/Round")

plt.savefig(
    "/content/gdrive/Shareddrives/AI Engineering/PETs/Privacy-Sobhan Hemati/Kidney/history_fed_train_id_0_2_3_result.jpg",
    bbox_inches="tight",
)